In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/pbl05-plant/ForPBL05/train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/pbl05-plant/ForPBL05/val',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
cnn = tf.keras.models.Sequential()

# Layer 1
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[128, 128, 3]))
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.BatchNormalization())

# Layer 2
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.BatchNormalization())

# Layer 3
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.BatchNormalization())

# Layer 4
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.BatchNormalization())

# Fully Connected Layer
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.4))

# Output Layer
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax'))

# Compile Model
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

cnn.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training the model
training_history = cnn.fit(
    x=training_set,
    validation_data=validation_set,
    epochs=50,  # You can start with 50 epochs and let early stopping handle the rest
    callbacks=[early_stopping]
)

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/pbl05-plant/ForPBL05/test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
#Training set Accuracy
train_loss, train_acc = cnn.evaluate(training_set)
print('Training accuracy:', train_acc)

In [ ]:
#Validation set Accuracy
val_loss, val_acc = cnn.evaluate(validation_set)
print('Validation accuracy:', val_acc)

In [ ]:
cnn.save('trained_plant_disease_model_PBL05.keras')

In [ ]:
cnn.save('trained_plant_disease_model_PBL05.h5')

In [ ]:
training_history.history

In [ ]:
#Recording History in json
import json
with open('training_hist.json','w') as f:
  json.dump(training_history.history,f)

In [ ]:
print(training_history.history.keys())

In [ ]:
epochs = [i for i in range(1,11)]
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel('No. of Epochs')
plt.title('Visualization of Accuracy Result')
plt.legend()
plt.show()

**Đây là để Test lại :**

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/pbl05-plant/ForPBL05/test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

In [ ]:
y_pred = cnn.predict(test_set)
predicted_categories = tf.argmax(y_pred, axis=1)

In [ ]:
true_categories = tf.concat([y for x, y in test_set], axis=0)
Y_true = tf.argmax(true_categories, axis=1)

In [ ]:
Y_true

In [ ]:
predicted_categories

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(Y_true,predicted_categories)

In [ ]:
class_name = test_set.class_names

In [ ]:
# Precision Recall Fscore
print(classification_report(Y_true,predicted_categories,target_names=class_name))

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(cm,annot=True,annot_kws={"size": 10})

plt.xlabel('Predicted Class',fontsize = 20)
plt.ylabel('Actual Class',fontsize = 20)
plt.title('Plant Disease Prediction Confusion Matrix',fontsize = 10)
plt.show()

**Này để trên web**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/pbl05-plant/ForPBL05/val',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)
class_name = validation_set.class_names
print(class_name)

In [ ]:
cnn = tf.keras.models.load_model('trained_plant_disease_model_PBL05.keras')

In [ ]:
#Test Image Visualization
import cv2
image_path = '/kaggle/input/pbl05-plant/ForPBL05/test/Healthy/photo36.jpg'
# Reading an image in default mode
img = cv2.imread(image_path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Converting BGR to RGB
# Displaying the image 
plt.imshow(img)
plt.title('Test Image')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
image = tf.keras.preprocessing.image.load_img(image_path,target_size=(128,128))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
predictions = cnn.predict(input_arr)

In [ ]:
print(predictions)

In [ ]:
result_index = np.argmax(predictions) #Return index of max element
print(result_index)

In [ ]:
# Displaying the disease prediction
model_prediction = class_name[result_index]
plt.imshow(img)
plt.title(f"Disease Name: {model_prediction}")
plt.xticks([])
plt.yticks([])
plt.show()

In [1]:
from tensorflow.keras.models import load_model

# Load mô hình đã lưu
cnn = load_model('/kaggle/input/qqqq/keras/aaas/2/trained_plant_disease_model_PBL05.keras')


2024-06-12 03:58:34.812461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 03:58:34.812653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 03:58:35.016271: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
test_set = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/pbl05-plant/ForPBL05/test',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,  # Để dữ liệu không bị xáo trộn
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)


Found 230 files belonging to 3 classes.


In [5]:
test_loss, test_acc = cnn.evaluate(test_set)
print('Test accuracy:', test_acc)


I0000 00:00:1718164943.325861      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-06-12 04:02:23.329293: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.


8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 376ms/step - accuracy: 0.8739 - loss: 0.2639
Test accuracy: 0.9304347634315491


In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Lấy giá trị thực tế và giá trị dự đoán từ tập kiểm tra
y_true = []
y_pred = []

for images, labels in test_set:
    predictions = cnn.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(predictions, axis=1))

# Tính RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f'RMSE: {rmse}')

# Tính MAPE
mape = mean_absolute_percentage_error(y_true, y_pred)
print(f'MAPE: {mape}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 583ms/step
RMSE: 0.3296902366978935
MAPE: 234970415341069.34


In [7]:
import time

# Đo thời gian suy diễn trên một batch
def measure_inference_time(model, dataset, batch_size):
    total_time = 0
    num_batches = 0
    for images, _ in dataset.take(10):  # lấy 10 batch để đo
        start_time = time.time()
        _ = model.predict(images)
        end_time = time.time()
        total_time += (end_time - start_time)
        num_batches += 1
    avg_time_per_batch = total_time / num_batches
    avg_time_per_image = avg_time_per_batch / batch_size
    return avg_time_per_image

batch_size = 32
avg_time_per_image = measure_inference_time(cnn, test_set, batch_size)
fps = 1 / avg_time_per_image

print(f'Average inference time per image: {avg_time_per_image:.6f} seconds')
print(f'FPS: {fps:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 648ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Average inference time per image: 0.013050 seconds
FPS: 76.63


In [8]:
# Giả lập dữ liệu về Uptime và MTBF
uptime_hours = 1000  # Giả lập hệ thống đã hoạt động 1000 giờ
num_failures = 5     # Giả lập có 5 lần hỏng hóc

# Tính MTBF
mtbf = uptime_hours / num_failures
print(f'MTBF: {mtbf} hours')

# Tính Uptime
total_time = 24 * 30  # Tổng thời gian trong 1 tháng (giả sử)
uptime_percentage = (uptime_hours / total_time) * 100
print(f'Uptime: {uptime_percentage:.2f}%')


MTBF: 200.0 hours
Uptime: 138.89%
